In [ ]:
#Import the required libraries
import os
import time
import sys
import pathlib
import glob
import stat
import gzip
import shutil
import pandas as pd
from datetime import datetime
from functools import wraps
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, WebDriverException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
from chromedriver_py import binary_path

#os.chdir('D:\\Projects\\Webscraping 001\\TRADINGVIEW')
os.chdir(r'D:\TD\FMF\Strategies')

A <b>PRO Tradingview </b>account is needed in order to get charts with timeframes of less than a minute. 
I am using my Chrome saved data to access the account without going through the login page.
I am also changing the <b>default download folder</b>.

In [2]:
# Configurations
user = 'silvi'
os.environ['USER_DATA_DIR'] = r'C:\\Users\\{}\\AppData\\Local\\Google\\Chrome\\User Data'.format(user)
os.environ['DEFAULT_DOWNLOAD_DIR'] = 'D:\\TD\\FMF\\Strategies\\temp'
os.environ['CHART_URL'] = 'https://www.tradingview.com/chart/h9CR9nMd/'

service = Service(executable_path=binary_path)

options = webdriver.ChromeOptions()

options.add_argument('--user-data-dir=C:\\Users\\{}\\AppData\\Local\\Google\\Chrome\\User Data'.format(user))
options.add_argument("--start-maximized")
options.add_experimental_option("prefs", {
    "download.default_directory": os.getenv('DEFAULT_DOWNLOAD_DIR'),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})


options.set_capability("goog:chromeOptions", DesiredCapabilities.CHROME.copy())

##########################################################################################################

driver = webdriver.Chrome(service=service, options=options)

driver.implicitly_wait(1)


# Open URL
for _ in range(2):
    try:
        driver.get(os.getenv('CHART_URL'))
        print('Chart opened successfully')
        break
    except WebDriverException:
        time.sleep(5)

wait = WebDriverWait(driver, 5)

Chart opened successfully


<b>Tradingview</b> uses a rotation scheme so that the relative XPATH I am using is not 'stable' but has several options. The Rotation is stored in a <div> tag. 
Here, I am getting the rotation number and I confirm that it works.

In [3]:
######################################### PRELIMINARY FUNCTIONS

##########################################  REPEAT FUNCTION
def retry(max_tries=3, delay_seconds=2):
    def decorator_retry(func):
        @wraps(func)
        def wrapper_retry(*args, **kwargs):
            tries = 0
            while tries < max_tries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    tries += 1
                    if tries == max_tries:
                        raise e
                    time.sleep(delay_seconds)
        return wrapper_retry
    return decorator_retry

#######################################  GET THE ROTATION NUMBER FOR THE CHANGING ELEMENTS
rotation = None  # Initialize rotation outside the function

#This list is used to get the rotation version. It is the list of the selectors of the dropdown menu for the strategy selection
l1 = ['//*[@id="bottom-area"]/div[3]/div/div[1]/div[1]/div[1]/button',
    '//*[@id="bottom-area"]/div[4]/div/div[1]/div[1]/div[1]/button',
    '//*[@id="bottom-area"]/div[5]/div/div[1]/div[1]/div[1]/button',
     '//*[@id="bottom-area"]/div[6]/div/div[1]/div[1]/div[1]/button']

@retry(max_tries=3, delay_seconds=2)
def getTheRotationNumber(whichElement):
    try:
        #Here we open the bottom TAB to see the Strategy Tester
        driver.find_element(By.XPATH, '//*[@id="footer-chart-panel"]/div[1]/div[1]/div[3]').click()
    except Exception as e1:
        try:
            driver.find_element(By.XPATH, '//*[@id="footer-chart-panel"]/div[1]/div[1]/div[3]/div/span').click()
        except Exception as e2:
            pass

    rotation = next((idx for idx, b in enumerate(whichElement) if driver.find_element(By.XPATH, b)), None)
    print("Found rotation:", rotation)
    
    #this is risky because it can be an endless loop. I put it however in case the TAB was closed and I need the script to reopen it
    while rotation is None:
        continue

    return rotation


rotation = getTheRotationNumber(l1)

Found rotation: 0


<b>These are just the lists of elements which change because of the rotation. </b>

In [5]:
###############################SPECIFY THE VERSIONS OF THE LISTS TO USE

l2 = ['//*[@id="bottom-area"]/div[3]/div/div[1]/div[1]/div[1]/button/span[3]',
    '//*[@id="bottom-area"]/div[4]/div/div[1]/div[1]/div[1]/button/span[3]',
    '//*[@id="bottom-area"]/div[5]/div/div[1]/div[1]/div[1]/button/span[3]',
     '//*[@id="bottom-area"]/div[6]/div/div[1]/div[1]/div[1]/button/span[3]']

#list for the strategy selector
l2a = ['//*[@id="bottom-area"]/div[3]/div/div[1]/div[1]/div[1]/button/span[2]',
    '//*[@id="bottom-area"]/div[4]/div/div[1]/div[1]/div[1]/button/span[2]',
    '//*[@id="bottom-area"]/div[5]/div/div[1]/div[1]/div[1]/button/span[2]',
      '//*[@id="bottom-area"]/div[6]/div/div[1]/div[1]/div[1]/button/span[2]']

#list of No trades
l3 = ['//*[@id="bottom-area"]/div[3]/div/div[2]/div[1]/div[2]', 
      '//*[@id="bottom-area"]/div[4]/div/div[2]/div[1]/div[2]',
      '//*[@id="bottom-area"]/div[5]/div/div[2]/div[1]/div[2]',
     '//*[@id="bottom-area"]/div[6]/div/div[2]/div[1]/div[2]']

#list of download csv button selector
l4 = ['//*[@id="bottom-area"]/div[3]/div/div[1]/div[1]/div[1]/div/button[3]',
      '//*[@id="bottom-area"]/div[4]/div/div[1]/div[1]/div[1]/div/button[3]',
      '//*[@id="bottom-area"]/div[5]/div/div[1]/div[1]/div[1]/div/button[3]',
     '//*[@id="bottom-area"]/div[6]/div/div[1]/div[1]/div[1]/div/button[3]']

<b>These are various functions used while interacting with the page</b>

In [6]:
#functions
#files and temp dirs
fncsv = 'StrategyTestingResults v2.csv.gz'
tempdir = pathlib.Path(r'D:\TD\FMF\Strategies\temp')
tdir = r'D:\TD\FMF\Strategies\temp'

#Function to delete the temporary downloaded files
def deleter(d):
    for f in os.listdir(d):
        try:
            os.chmod(f,stat.S_IRUSR|stat.S_IRGRP|stat.S_IROTH|stat.S_IXUSR|stat.S_IRUSR|stat.S_IWUSR|stat.S_IWGRP|stat.S_IXGRP)
            os.remove(os.path.join(d, f))
        except:
            pass

#Function to identify the last csv file downloaded
def getlastcsv(l):
    files = glob.glob(l) # * means all if need specific format then *.csv
    p = max(files, key=os.path.getctime)
    return(p)

#Retry function when the Rotation is changed during script execution
def dl(whichList):
    global rotation
    try: 
        driver.find_element(By.XPATH, whichList[rotation]).click() 
    except:
        rotation = getTheRotationNumber()
        driver.find_element(By.XPATH, whichList[rotation]).click()                     
    time.sleep(2)
    try:
        #lastcsv = tempdir / '*.csv'
        lastcsv = r'D:\TD\FMF\Strategies\temp\*.csv'
        filepath = getlastcsv(lastcsv)
    except ValueError:
        time.sleep(2)
        return None
        
    return(filepath)

#Preprocessing every downloaded file, returns a df
def processFile(filepath, results, t, timeframe, strategy):
    totalres = pd.DataFrame()
    
    if filepath is None:
        print('Filepath is empty')
        return results
    
    try: 
        data = pd.read_csv(filepath, low_memory=False, parse_dates=['Date/Time'], dayfirst=True, index_col=None)
        data = data.dropna()
        data['Date/Time'] = pd.to_datetime(data['Date/Time'], dayfirst=True)
        data['ticker'] = t
        data['timeframe'] = timeframe
        data['strategy'] = strategy
        results = pd.concat([results, data], ignore_index=True, axis=0)
        
    except Exception:
        print('There is an error when processing the data')
    return results

#function to send the characters for each PAIR to the browser
def send_keys_with_retry(element, text):
    max_retries = 3
    retries = 0
    while retries < max_retries:
        try:
            element.send_keys(Keys.CONTROL + "a")  # Select all contents
            element.send_keys(Keys.BACKSPACE)  # Clear contents
            element.send_keys(text)
            return
        except StaleElementReferenceException:
            retries += 1
            time.sleep(1)

<b>And this is the main extract function looping through timeframe and strategy. The ticker loop was moved outside to be able to save the data after each ticker loop</b>

In [8]:
##################################### DEFINE THE MAIN EXTRACT FUNCTION.
def extract(t, tm, sl):
    global rotation
    results = pd.DataFrame()
    output = pd.DataFrame()
    data = pd.DataFrame()

    #ticker block - looping each when executing 
    ticker = driver.find_element(By.XPATH, '//*[@id="header-toolbar-symbol-search"]')
    
    #print(t)# - FOR CHECKING
    
    #send the ticker to the browser
    with webdriver.common.action_chains.ActionChains(driver) as action:
        action.move_to_element(ticker).click()
        for i in t:
            send_keys_with_retry(action, i)
        action.send_keys(Keys.RETURN).perform()

    #timeframe block loop
    for ii in tm:
        tt = driver.find_element(By.XPATH, '//*[@id="header-toolbar-intervals"]/button') #click on the timeframe icon
        driver.execute_script("$(arguments[0]).click();", tt)
        time.sleep(1)
        tlr = driver.find_element(By.XPATH, ii) #select the timeframe
        driver.execute_script("$(arguments[0]).click();", tlr)

        timeframe = driver.find_element(By.XPATH, '//*[@id="header-toolbar-intervals"]/button').text #save the timeframe
        time.sleep(3)

        outputs = []  # Initialize an empty list to store output DataFrames
        
        #strategy block loop
        for s in sl:
            st = driver.find_element(By.XPATH, l2[rotation])
            driver.execute_script("$(arguments[0]).click();", st)
            time.sleep(1)
            stl = driver.find_element(By.XPATH, s) # Select each strategy
            driver.execute_script("$(arguments[0]).click();", stl)
            strategy = driver.find_element(By.XPATH, l2a[rotation]).text
            
            #Skip if no trades
            NoTrades = ''
            try:
                try:
                    NoTrades = driver.find_element(By.CLASS_NAME, 'text-yrIMi47q').text
                except:
                    rotation = getTheRotationNumber()
                    NoTrades = driver.find_element(By.XPATH, l3[rotation]).text              
            except:
                NoTrades = '0'

            if NoTrades == 'No data':
                pass
            
            else:
                os.chdir(tdir)
                lastcsv = r'D:\TD\FMF\Strategies\temp\*.csv'
                deleter(tdir)
                
                try:
                    filepath = dl(l4)

                    while filepath is None or not os.path.isfile(filepath):
                        time.sleep(5)
                        filepath = dl(l4)

                    if os.path.isfile(filepath):
                        output = processFile(filepath, results, t, timeframe, strategy)
                        output.columns = ['Trade #', 'Type', 'exit_signal', 'exit_time', 'exit_price', 'contracts', 'profit', 'profit%', 
                                           'Cum. Profit', 'Cum. Profit %', 'runup', 'runup%', 'drawdown', 'drawdown%', 'ticker', 
                                           'timeframe', 'strategy']
                        cols0 = ['Trade #', 'ticker', 'timeframe', 'strategy', 'exit_signal', 'exit_time', 'exit_price', 'profit%', 'runup%', 'drawdown%']
                        output = output.reindex(columns=cols0)
                        time.sleep(5)
                        outputs.append(output)  # Append output DataFrame to the list
                except Exception:
                    print('Some error')
    time.sleep(2)
    totalOutput = pd.concat(outputs, ignore_index=True)  # Concatenate all output DataFrames at once
    return totalOutput

<b>In Tradingview, each complete trade defined by a cycle BUY-SELL takes 2 rows because they are TWO separate trades. <br>
I combine the data into one row to be able to assess the profitability per complete trade, not onl for each segment of the trade.<br>
I then delete the extra-rows and reverse the table.<br>
VERY IMPORTANT: All the strategies are set to trade a fixed amount which is 10,000 USD. This is to standardise the profit, runup and drawdown.</b>

In [9]:
#Function to restructure the data
def transform(results):
    
    try:
        results = results.assign(entry_price=None)
        results = results.assign(entry_time=None)
        results['entry_signal'] = ''
        
        # Convert the entry_price and exit_price columns to float64
        results['entry_price'] = results['entry_price'].astype('float64')
        results['exit_price'] = results['exit_price'].astype('float64')

        # Convert the entry_time and exit_time columns to datetime64[s]
        results['entry_time'] = results['entry_time'].astype('datetime64[s]')
        results['exit_time'] = results['exit_time'].astype('datetime64[s]')

        results['entry_time'] = pd.to_datetime(results['entry_time'])#, dayfirst=True)
        results['exit_time'] = pd.to_datetime(results['exit_time'])#, dayfirst=True)
        
        # Select the relevant columns
        cols = ['ticker', 'timeframe', 'strategy', 'entry_price', 'entry_time', 'entry_signal', 'exit_price', 'exit_time', 'exit_signal', 
                'runup%', 'drawdown%']
        results = results.reindex(columns=cols)

        ########################################################################################################################################
        # Create a new DataFrame with the transformed values
        r = results.iloc[::-1]
        r['entry_price'] = r['exit_price']
        r['entry_time'] = r['exit_time']
        r['entry_signal'] = r['exit_signal']
        r['exit_price'] = r['exit_price'].shift(-1)
        r['exit_time'] = r['exit_time'].shift(-1)
        r['exit_signal'] = r['exit_signal'].shift(-1)
        res = r.iloc[::2].copy()

        ########################################################################################################################################
            
        # Use the function to transform the DataFrame

        res = res.drop_duplicates(subset=['ticker', 'timeframe', 'strategy', 'entry_price', 'entry_time', 
                                          'exit_price', 'exit_time'], keep='last', inplace=False)

        
    except Exception as e:
        print(e)
        res = pd.DataFrame()
    
    return res

<b>Function to save the transformed data & append it to the existing gzipped file</b>

In [10]:
#Function to save the transformed data & append it to the existing gzipped file
def savecsv(df, file):
    os.chdir(r'D:\TD\FMF\Strategies')
    if os.path.isfile(file):
        # If file exists, append the DataFrame to the existing file
        with gzip.open(file, 'at', compresslevel=5) as ffile:
            df.to_csv(ffile, header=False, index=False)
    else:
        # If file does not exist, save the DataFrame to a new file
        df.to_csv(file, index=False, compression='gzip')
    return

<b>This is where you can input your own tickers, timeframes and strategies.</b>

In [11]:
#List of tickers
ticker = ['BINANCE:BTCUSDT.P', 
           'BINANCE:ETHUSDT.P', 
           'BINANCE:BNBUSDT.P',
        ]

#List of timeframes - SELECT YOURS
tf = ['//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div/div[2]/div/span[1]/span',
       '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div/div[3]/div/span[1]/span',
           '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div/div[4]/div/span[1]/span',
           '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div/div[5]/div/span[1]/span',
           '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div/div[6]/div/span[1]/span',
           '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div/div[9]/div/span[1]/span']

strategy = ['//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[1]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[2]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[3]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[4]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[5]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[6]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[7]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[8]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[9]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[10]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[11]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[12]/span/span',
              '//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[13]/span/span']

<b>We finally run the code and get some data</b>

In [13]:
#################RUN THE STRATEGY
#Save the start time
start_time = datetime.now()
start_time_s = start_time.strftime("%d/%m/%Y %H:%M:%S")
print("Start Time =", start_time_s)    

collectedOutput = pd.DataFrame()
with tqdm(total=len(ticker), file=sys.stdout) as pbar:
    for tticker in ticker:
    
        localOutput = extract(tticker, tf, strategy)
        try:
            
            collectedOutput = collectedOutput.append(localOutput, ignore_index=True)
        
        except:
            collectedOutput = collectedOutput
        pbar.update(1)

finalOutput = transform(collectedOutput)
savecsv(finalOutput, fncsv)

###############################	Resave the file after cleaning empty rows and duplicates 
#This is to deal with the complications added by saving to gzip instead of simple csv
def cleanSave(file):
    tempOutputFile = 'cleaned_' + file  # New file to save cleaned data
    with gzip.open(file, 'rt') as f:
        os.chdir(r'D:\TD\FMF\Strategies')
        data = pd.read_csv(f, parse_dates=['entry_time', 'exit_time'], index_col=None)
    data = data.sort_values(['entry_time', 'exit_time'], ascending = (True, True))
    data['entry_time'] = pd.to_datetime(data['entry_time'], dayfirst=True)
    data['exit_time'] = pd.to_datetime(data['exit_time'], dayfirst=True)
    data = data.drop_duplicates(subset=['ticker', 'timeframe', 'strategy', 'entry_price', 'entry_time', 
                    'exit_price', 'exit_time'], keep='last', inplace=False)
    data.dropna(axis=0, how='all', inplace=True)
    return(data)

def replaceFile(data):
    # Save DataFrame to temporary uncompressed CSV file
    tempFile = 'temp.csv'
    data.to_csv(tempFile, index=False)
    
    with open(tempFile, 'rb') as f_in, gzip.open(file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

file = 'StrategyTestingResults v2.csv.gz'
data = cleanSave(file)
replaceFile(data)

#close the bottom tab
try:
    driver.find_element(By.XPATH, '//*[@id="footer-chart-panel"]/div[1]/div[1]/div[3]').click()
except Exception:
    try:
        driver.find_element(By.XPATH, '//*[@id="footer-chart-panel"]/div[1]/div[1]/div[3]/div/span').click()
    except Exception:
        pass

#save the end time
end_time = datetime.now()
end_time_s = end_time.strftime("%d/%m/%Y %H:%M:%S")
print("End Time =", end_time_s)

#Compute duration and print to screen
d = end_time - start_time
duration = d.total_seconds()
duration = duration / 60
print('The job took ' + str(duration) + ' minutes.')
time.sleep(5)

#Close the browser
driver.close()

Start Time = 12/10/2023 16:21:13
100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [45:36<00:00, 912.23s/it]
End Time = 12/10/2023 17:06:58
The job took 45.76011875 minutes.


In [14]:
data

,ticker,timeframe,strategy,entry_price,entry_time,entry_signal,exit_price,exit_time,exit_signal,runup%,drawdown%
0,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.777,2023-07-04 01:19:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.44,0.48
1,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.783,2023-07-04 01:19:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.35,0.56
2,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.786,2023-07-04 01:19:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.31,0.61
3,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.788,2023-07-04 01:25:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.28,0.64
4,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.787,2023-07-04 01:26:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.29,0.62
...,...,...,...,...,...,...,...,...,...,...,...
312249,BINANCE:BNBUSDT.P,1m,[CP] ChannelBreakOutStrategy,205.470,2023-10-12 14:52:00,SELL,205.570,2023-10-12 14:56:00,BUY,0.07,0.05
312250,BINANCE:BNBUSDT.P,1m,[CP] ChannelBreakOutStrategy,205.570,2023-10-12 14:56:00,BUY,205.500,2023-10-12 15:02:00,SELL,0.04,0.04
314451,BINANCE:BNBUSDT.P,1m,[CP] Momentum Strategy,205.590,2023-10-12 14:57:00,BUY,205.590,2023-10-12 15:00:00,SELL,0.03,0.02
314452,BINANCE:BNBUSDT.P,1m,[CP] Momentum Strategy,205.590,2023-10-12 15:00:00,SELL,205.550,2023-10-12 15:05:00,BUY,0.13,0.00
